In [4]:
#Kernal PCA jupyter notebook 
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split

train = pd.read_csv("../input/train_dummy.csv", header=0, index_col=0)
targets = pd.read_csv("../input/y_train.csv", header = 0, index_col = 0)

X = train.values
y = targets.values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(6505, 920)
(2788, 920)
(6505, 1)
(2788, 1)


In [35]:
#Incremental kernal PCA
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
pca = PCA()
pca.fit(X)
cumsum = np.cumsum(pca.explained_variance_ratio_)
d = np.argmax(cumsum >= 0.99) + 1

pca = PCA(n_components = d)
X_train_red = pca.fit_transform(X_train)
X_test_red = pca.fit_transform(X_test)
l1 = LGBM.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)
l2 = LGBM.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720, 
                              max_bin = 55, bagging_fraction = 0.8, 
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)

l1.fit(X_train, y_train)
l2.fit(X_train_red, y_train)

pred1 = l1.predict(X_test)
pred2 = l2.predict(X_test_red)
print(X_test_red.shape)
print(y_test.shape)

print("model 1 acc: ", mean_squared_error(pred1, y_test))

print("model 2 acc: ", mean_squared_error(pred2, y_test))

C:\Users\srder\Miniconda3\lib\site-packages\sklearn\utils\validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\srder\Miniconda3\lib\site-packages\sklearn\utils\validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(2788, 93)
(2788, 1)
model 1 acc:  0.00623149347959
model 2 acc:  0.0272553392157


In [20]:
#Grid Search Light GBM
import lightgbm as LGBM
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.decomposition import KernelPCA

kernels = ['rbf', 'sigmoid']

kernel_clf = Pipeline([('KPCA', KernelPCA(n_components = 900, n_jobs = -1)),
                       ('lgbm', LGBM.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720, 
                              max_bin = 55, bagging_fraction = 0.8, 
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11))])

param_grid = [{
    'KPCA__gamma': np.linspace(0.9, 1, 10),
    'KPCA__kernel': kernels
}]

grid_search = GridSearchCV(kernel_clf, param_grid, cv = 3)
grid_search.fit(X, y)
grid_search.best_params_

C:\Users\srder\Miniconda3\lib\site-packages\sklearn\decomposition\kernel_pca.py:264: RuntimeWarning: invalid value encountered in sqrt
  X_transformed = self.alphas_ * np.sqrt(self.lambdas_)
C:\Users\srder\Miniconda3\lib\site-packages\sklearn\utils\validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\srder\Miniconda3\lib\site-packages\sklearn\decomposition\kernel_pca.py:285: RuntimeWarning: invalid value encountered in sqrt
  return np.dot(K, self.alphas_ / np.sqrt(self.lambdas_))
C:\Users\srder\Miniconda3\lib\site-packages\sklearn\decomposition\kernel_pca.py:264: RuntimeWarning: invalid value encountered in sqrt
  X_transformed = self.alphas_ * np.sqrt(self.lambdas_)
C:\Users\srder\Miniconda3\lib\site-packages\sklearn\utils\validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. 

LightGBMError: b'Cannot construct Dataset since there are not useful features.                 It should be at least two unique rows.                 If the num_row (num_data) is small, you can set min_data=1 and min_data_in_bin=1 to fix this.                 Otherwise please make sure you are using the right dataset.'

In [8]:
#Grid Search Light GBM
import lightgbm as LGBM
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint

lgm = LGBM.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720, 
                              max_bin = 55, bagging_fraction = 0.8, 
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)

param_dist = {'learning_rate': np.linspace(0.01, 0.1, 10),
              'max_bins': sp_randint(50, 60),
              'baggin_frequency': sp_randint(1, 10),
              'baggin_fraction': np.linspace(0.2, 0.3, 10),
              'feature_fraction': np.linspace(0.2,0.3,10),
              'min_data_in_leaf': sp_randint(3,10),
              'num_leaves': sp_randint(4, 10)}

param_grid = [{
    'learning_rate': np.linspace(0.01, 0.1, 10),
    'max_bins': np.linspace(50, 60, 10),
    'baggin_frequency': np.linspace(1,10,10),
    'feature_fraction': np.linspace(0.2,0.3, 10),
}]

n_iter = 40
rand_search = RandomizedSearchCV(lgm, param_distributions=param_dist, cv = 3)
rand_search.fit(X, y)
rand_search.cv_results_

C:\Users\srder\Miniconda3\lib\site-packages\sklearn\utils\validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\srder\Miniconda3\lib\site-packages\sklearn\utils\validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\srder\Miniconda3\lib\site-packages\sklearn\utils\validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\srder\Miniconda3\lib\site-packages\sklearn\utils\validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,

KeyboardInterrupt: 